# Qwen2.5-7B模型微调流程
本notebook参考Qwen2_5_7B_Alpaca_fintune.ipynb，使用已处理好的QA数据进行微调。

## 2. 加载和配置Qwen2.5-7B模型
设置最大序列长度、数据类型和量化方式，加载预训练模型和分词器。

## 3. 添加LoRA适配器
为模型添加LoRA适配器，只微调部分参数以节省显存和加速训练。

## 4. 加载QA数据集
加载已处理好的QA数据集（qa_train.json），用于微调。

## 5. 格式化数据并添加EOS标记
定义格式化函数，将指令、输入和输出拼接为训练文本，并在末尾添加EOS标记。

## 6. 训练模型
使用TRL的SFTTrainer进行微调，设置训练参数如批次大小、学习率、训练步数等。

## 7. 显存信息监控
通过torch.cuda获取和打印显卡显存使用情况，便于监控资源消耗。

## 8. 启动训练并监控显存变化
开始训练模型，并显示训练结果和显存变化。

## 9. 推理与保存微调后的模型
训练完成后，进行简单推理验证，并保存LoRA适配器和分词器。

In [ ]:
import torch
from unsloth import FastLanguageModel
# 确保trl库已安装：pip install trl
from trl import SFTConfig, SFTTrainer
# 确保datasets库已安装：pip install datasets
from datasets import Dataset
import json
from sklearn.metrics import recall_score
import numpy as np

# ===================== 显存计算工具函数 =====================
def calculate_vram_usage(batch_size, seq_len):
    """
    计算FP16精度下的显存占用（适用于LoRA微调场景）
    公式：总占用 = 模型权重(14GB) + KV缓存(batch_size×seq_len×4×2/1024³) + 梯度/优化器(LoRA下可忽略)
    梯度检查点节省约40%显存
    """
    model_weight = 14  # 7B模型FP16权重约14GB
    kv_cache = batch_size * seq_len * 4 * 2 / (1024**3)  # 4层注意力机制，FP16精度
    gradient_checkpoint = 0.6  # 梯度检查点节省40%显存
    return round((model_weight + kv_cache) * gradient_checkpoint, 2)


# ===================== 1. 金融场景训练配置预设 =====================
finance_sft_config = {
    "learning_rate": 1e-5,    # 金融数据敏感，采用小学习率避免过拟合
    "num_train_epochs": 3,    # 控制训练轮数，平衡收敛与过拟合
    "per_device_train_batch_size": 16,
    "lora_r": 16,             # 适配金融领域知识的LoRA秩
    "max_seq_length": 2048,   # 金融文本较长，设置合理序列长度
}

# 提前计算显存占用并提示
predicted_vram = calculate_vram_usage(
    finance_sft_config["per_device_train_batch_size"],
    finance_sft_config["max_seq_length"]
)
print(f"预估显存占用: {predicted_vram} GB")


# ===================== 2. 加载和配置Qwen2.5-7B模型 =====================
max_seq_length = finance_sft_config["max_seq_length"]
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # 如需访问受限模型请填写token
)


# ===================== 3. 添加金融场景适配的LoRA适配器 =====================
model = FastLanguageModel.get_peft_model(
    model,
    r = finance_sft_config["lora_r"],
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)


# ===================== 4. 加载金融SFT数据集（与spark_data_process.ipynb输出匹配） =====================
qa_data_path = "./datas/finance_sft_train.json"
with open(qa_data_path, "r", encoding="utf-8") as f:
    qa_data = json.load(f)

# 转换为Hugging Face Dataset
dataset = Dataset.from_list(qa_data)
print(f"金融SFT样本数: {len(dataset)}")
print("示例金融样本:", dataset[0])


# ===================== 5. 格式化数据并添加EOS标记（适配金融指令格式） =====================
EOS_TOKEN = tokenizer.eos_token
finance_prompt = """你是专业金融分析师助手，需基于研报、财报等金融文档内容回答，严格区分事实与观点，遵循金融术语规范。\n\n### 指令:\n{instruction}\n\n### 输出:\n{output}"""

def formatting_finance_prompts(examples):
    texts = []
    for inst, out in zip(examples["instruction"], examples["output"]):
        formatted = finance_prompt.format(instruction=inst, output=out) + EOS_TOKEN
        texts.append(formatted)
    return {"text": texts}

dataset = dataset.map(formatting_finance_prompts, batched=True)


# ===================== 6. 初始化金融场景SFT训练器 =====================
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False,
    args = SFTConfig(
        per_device_train_batch_size = finance_sft_config["per_device_train_batch_size"],
        num_train_epochs = finance_sft_config["num_train_epochs"],
        learning_rate = finance_sft_config["learning_rate"],
        warmup_steps = 10,
        max_steps = -1,  # 由num_train_epochs自动控制
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",  # 余弦调度更适配金融数据的收敛
        seed = 3407,
        output_dir = "spark_finance_model_outputs",
        report_to = "none",
    ),
)


# ===================== 7. 显存信息监控 =====================
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU型号: {gpu_stats.name}，最大显存: {max_memory} GB")
print(f"已预留显存: {start_gpu_memory} GB")


# ===================== 8. 启动金融场景模型训练 =====================
trainer_stats = trainer.train()
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"训练耗时: {trainer_stats.metrics['train_runtime']} 秒")
print(f"训练耗时: {round(trainer_stats.metrics['train_runtime']/60, 2)} 分钟")
print(f"训练期间峰值显存: {used_memory} GB，占最大显存 {used_percentage}%")
print(f"LoRA训练显存: {used_memory_for_lora} GB，占最大显存 {lora_percentage}%")


# ===================== 9. 跨集验证：纯新数据评估Recall =====================
# 加载纯新验证集（未参与训练/检索的数据）
val_data_path = "./datas/finance_sft_val.json"  # 需提前准备纯新验证集
if os.path.exists(val_data_path):
    with open(val_data_path, "r", encoding="utf-8") as f:
        val_data = json.load(f)
    val_dataset = Dataset.from_list(val_data)
    print(f"\n纯新验证集样本数: {len(val_dataset)}")
    print("验证集示例样本:", val_dataset[0])

    # 格式化验证集数据
    def format_val_prompts(examples):
        texts = []
        for inst in examples["instruction"]:
            formatted = finance_prompt.format(instruction=inst, output="") + EOS_TOKEN
            texts.append(formatted)
        return {"text": texts, "label": examples["output"]}  # 保留真实标签用于Recall计算

    val_dataset = val_dataset.map(format_val_prompts, batched=True)

    # 推理并评估Recall
    all_preds = []
    all_labels = []
    for idx in range(len(val_dataset)):
        prompt = val_dataset[idx]["text"]
        label = val_dataset[idx]["label"]
        input_ids = tokenizer([prompt], return_tensors="pt").to("cuda")
        outputs = model.generate(**input_ids, max_new_tokens=128, use_cache=True)
        pred = tokenizer.batch_decode(outputs)[0].strip()

        # 文本匹配判断（简单示例：可根据业务需求替换为语义相似度计算）
        pred_match = 1 if label in pred else 0
        all_preds.append(pred_match)
        all_labels.append(1)  # 正例标签（假设所有验证样本都是需要召回的目标）

    # 计算Recall
    recall = recall_score(all_labels, all_preds)
    print(f"\n跨集验证Recall: {recall:.4f}")
else:
    print("\n警告：未找到纯新验证集文件，请准备finance_sft_val.json后重新运行验证步骤")


# ===================== 10. 金融场景推理与模型保存 =====================
FastLanguageModel.for_inference(model)  # 开启高效推理模式

# 测试金融问答能力
test_instruction = "联邦制药2024年营业收入和净利润分别是多少？"
test_prompt = finance_prompt.format(instruction=test_instruction, output="")
input_ids = tokenizer([test_prompt], return_tensors="pt").to("cuda")
outputs = model.generate(**input_ids, max_new_tokens=128, use_cache=True)
print("金融问答推理结果:", tokenizer.batch_decode(outputs))

# 保存金融LoRA适配器和分词器
model.save_pretrained("finance_lora_model")
tokenizer.save_pretrained("finance_lora_model")